In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import os
from sklearn.model_selection import train_test_split
import json

from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D

np.random.seed(1337)

%matplotlib inline

print("Modules loaded")

In [ ]:
# Normalize the input images and resize to a manageable size
alpha = -0.5
beta = 0.5
x_size = 32
y_size = 16

def preprocess_image(image):
    copy_image = np.uint8(image)
    norm_image = cv2.normalize(copy_image, copy_image, alpha=alpha, beta=beta, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    
    out_image = cv2.resize(norm_image, (x_size, y_size), interpolation=cv2.INTER_AREA)
    
    #if(flatten_input):
    #    out_image = np.array(out_image, dtype=np.float32).flatten()
    
    return out_image

def preprocess(array):
    normalized = []
    
    for i in range(len(array)):
        normalized.append(preprocess_image(array[i]))
                    
    return np.array(normalized)

In [ ]:
def extract_data_from_folder(folder_path):
    image_paths = []
    image_data = []
    steering_angles = []

    driving_log = np.genfromtxt(os.path.join(folder_path, 'driving_log.csv'), delimiter=',', dtype="S256,S256,S256,f8,f8,f8,f8")
    base_name = os.path.basename(folder_path)
    for record in driving_log:
        center_image = record[0].decode("utf-8")
        image_rel_path = center_image.rsplit(base_name + '/')[1]
        image_full_path = os.path.join(folder_path, image_rel_path)

        steering_angle = record[3]

        image = mpimg.imread(image_full_path)
        
        image_paths.append(image_full_path)
        image_data.append(image)
        steering_angles.append(steering_angle)
    
    return np.array(image_paths), np.array(image_data), np.array(steering_angles)

folder_list = ['data/t1', 'data/t1_extra']

print("Loading data from {0} folders:".format(len(folder_list)))

image_paths = []
image_data = []
steering_angles = []

image_shape = None

for folder_path in folder_list:
    image_paths_cur, image_data_cur, steering_angles_cur = extract_data_from_folder(folder_path)
    
    image_paths.append(image_paths_cur)
    image_data.append(image_data_cur)
    steering_angles.append(steering_angles_cur)
    
    num_training = len(image_paths_cur)
    
    if(image_shape == None):
        image_shape =  image_data_cur[0].shape
    
    assert(image_data_cur[0].shape == image_shape), "Data from folders need to have the same shape"
    
    print("  Folder '{0}': Training examples={1}, Image shape={2}".format(folder_path, num_training, image_data_cur[0].shape))

print("Done")

In [ ]:
print("Preprocessing input data...")

image_data_proc = []
for image_folder_data in image_data:
    image_data_proc_cur = preprocess(image_folder_data)
    image_data_proc.append(image_data_proc_cur)

print('Normalized images and resized data from a shape of', image_data[0][0].shape, 'to a shape of', image_data_proc[0][0].shape)

In [ ]:
%matplotlib inline

num_folders = len(folder_list)
for i in range(num_folders):
    subplot = plt.subplot(num_folders, 2, 2*i+1)
    subplot.imshow(image_data[i][0])
    subplot.set_title(image_paths[i][0])

    subplot = plt.subplot(num_folders, 2, 2*i+2)
    subplot.imshow(image_data_proc[i][0])
    #subplot.set_title(image_paths_1[0])
#plt.tight_layout()
plt.show()

for i in range(num_folders):
    subplot = plt.subplot(num_folders, 1, i+1)
    subplot.plot(steering_angles[i], 'b')
    subplot.set_xlim([0, len(steering_angles[i])])
    subplot.set_ylim([-1, 1])
    subplot.set_title("Steering Angle for '{0}' (degrees)".format(folder_list[i]))
plt.tight_layout()
plt.show()

In [ ]:
print("Merging data from all folders...")

X_train = np.concatenate((image_data_proc))
y_train = np.concatenate((steering_angles))

print("Size of training data is now: {0}".format(len(X_train)))

In [ ]:
print("Augmenting data with flipped images and steering angles...")

initial_size = len(X_train)

extra_images = []
extra_angles = []
for i in range(initial_size):
    flipped_image = cv2.flip(X_train[i,:], 1)
    extra_images.append(flipped_image)
    extra_angles.append(-y_train[i])

subplot = plt.subplot(1, 2, 1)
subplot.imshow(X_train[0, :])
subplot.set_title("Original: {0} degrees".format(y_train[0]))
subplot = plt.subplot(1, 2, 2)
subplot.imshow(extra_images[0])
subplot.set_title("Flipped: {0} degrees".format(extra_angles[0]))

#plt.tight_layout()
plt.show()

subplot = plt.subplot(1, 2, 1)
subplot.hist(y_train, 50, histtype='stepfilled')
subplot.set_title("Before augmentation")
subplot.set_yscale('log')
    
X_train = np.concatenate((X_train, np.array(extra_images)))
y_train = np.concatenate((y_train, np.array(extra_angles)))

subplot = plt.subplot(1, 2, 2)
subplot.hist(y_train, 50, histtype='stepfilled')
subplot.set_title("After augmentation")
subplot.set_yscale('log')

#plt.tight_layout()
plt.show()

print("Size of training data is now: {0} ({1:5.2f}% of the initial size)".format(len(X_train), 100. * len(X_train) / initial_size))

In [ ]:
print("Splitting the training data into training, validation and test sets...")

n_total = len(X_train)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=314)
print("Training size={0} ({1:5.2f}%), Validation size={2} ({3:5.2f}%) Test size={4} ({5:5.2f}%)".
      format(len(X_train), (100. * len(X_train) / n_total),
             len(X_val),  (100. * len(X_val) / n_total),
             len(X_test), (100. * len(X_test) / n_total)))

assert(round(np.mean(X_train)) == 0), "The mean of the input data is: %f" % np.mean(X_train)
assert(math.isclose(np.min(X_train), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_train), 0.5, abs_tol=1e-5)), "The range of the input data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))

In [ ]:
num_rows_proc = X_train[0].shape[0]
num_cols_proc = X_train[0].shape[1]
num_channels_proc = X_train[0].shape[2]

input_shape = (num_rows_proc, num_cols_proc, num_channels_proc)
nb_filters = 32
kernel_size = (3, 3)
pool_size = (2, 2)
n_classes = 1

print("Building model with: input_shape={0}, nb_filters={1}, kernel_size={2} pool_size={3} nb_classes={4}".
      format(input_shape, nb_filters, kernel_size, pool_size, n_classes))

model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], border_mode='valid', input_shape=input_shape))
model.add(Activation('relu'))

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(256, name='fc1'))
model.add(Activation('relu'))

model.add(Dense(128, name='fc2'))
model.add(Activation('relu'))

model.add(Dense(64, name='fc3'))
model.add(Activation('relu'))

model.add(Dense(32, name='fc4'))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(n_classes, name='output'))

model.summary()

In [ ]:
batch_size = 128
nb_epoch = 10

print("Training with batch_size={0} and nb_epoch={1}...".format(batch_size, nb_epoch))

model.compile(loss='mean_squared_error', optimizer=Adam())

history = model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch, verbose=1,
                    validation_data=(X_val, y_val))

print("Training ended")

In [ ]:
print("Evaluating the model performance on the test set...")

score = model.evaluate(X_test, y_test, verbose=1)

print("Test cost:", score)

acc_plot = plt.subplot(212)
acc_plot.plot(np.arange(1, nb_epoch + 1), history.history['loss'], 'r', label='Training')
acc_plot.plot(np.arange(1, nb_epoch + 1), history.history['val_loss'], 'b', label='Validation')
acc_plot.set_xlim([0, nb_epoch + 1])
#acc_plot.set_ylim([0, 100])
#acc_plot.set_yscale('log')
acc_plot.legend(loc=1)
acc_plot.set_title('Cost')
plt.tight_layout()
plt.show()

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(json.dumps(model_json))
        
model.save_weights("model.h5")

print("Model and weights saved")